# 7 operate turtlebot3 using gamepad and web controller

## prepare turlebot3
You can use either `turtlebot3_simulator` (A.) or actual `turtlebot3` robot (B.).

### A. use "turtlebot3 simulator" on Ubuntu desktop

#### prepare Ubuntu desktop as a turlebot3 simulator 

1. install `telepresence`

```bash
turtlebot3@turtlebot3-fake:~$ curl -s https://packagecloud.io/install/repositories/datawireio/telepresence/script.deb.sh | sudo bash
turtlebot3@turtlebot3-fake:~$ sudo apt install --no-install-recommends telepresence
```

```bash
turtlebot3@turtlebot3-fake:~$ telepresence --version
0.92
```

#### launch turtlebot3 simulator

1. start a terminal on Ubuntu Desktop.
2. start port-forward to `ros-master`

```bash
turtlebot3@turtlebot3-fake:~/ros_ws$ kubectl port-forward $(kubectl get pods -l app=ros-master -o template --template "{{(index .items 0).metadata.name}}") 11311:11311
```
3. run telepresence shell

```bash
turtlebot3@turtlebot3-fake:~/ros_ws$ telepresence --run-shell
```
4. set `ROS_MASTER_URI` and `ROS_HOSTNAME`

```bash
@minikube|bash-4.3$ export ROS_MASTER_URI=http://localhost:11311
@minikube|bash-4.3$ export ROS_HOSTNAME=$(minikube ip)
```
5. launch turtlebot3_fake on telepresence shell

```bash
@minikube|bash-4.3$ roslaunch turtlebot3_fake turtlebot3_fake.launch
```

### B. use actual "turtlebot3" robot

#### prepare Ubuntu desktop as a turlebot3 simulator

1. prepare turtlebot3
1. clone repositories for this demo from github.
    * https://github.com/tech-sketch/fiware_ros_turtlebot3_bridge.git
    * https://github.com/tech-sketch/fiware_ros_turtlebot3_operator.git
1. make repositories using `catkin_make`.

#### setup cert file to connect MQTT broker of fiware

1. copy the `ca.crt` from local to Ubuntu desktop

```bash
$ scp ${PJ_ROOT}/secrets/ca.crt ${user}@${Ubuntu_desktop}:${ROS_WS}/src/fiware_ros_turtlebot3_bridge/secrets/ca.crt
```

#### setup the parameters of MQTT broker

1. ssh to turtlebot3.
1. setup `mqtt.yaml`

```bash
ros:$ cp src/fiware_ros_turtlebot3_bridge/config/mqtt.yaml.template src/fiware_ros_turtlebot3_bridge/config/mqtt.yaml
ros:$ vi src/fiware_ros_turtlebot3_bridge/config/mqtt.yaml
```

#### install python libraries for bridge node

1. ssh to turtlebot3.
1. install python libraries for bridge node

```bash
ros:$ pip install -r src/fiware_ros_turtlebot3_bridge/requirements/common.txt
```

#### run roscore

1. ssh to turtlebot3.
1. run `roscore`.

```bash
ros:$ roscore
```

#### launch node to bridge fiware and ROS

1. ssh to turtlebot3.
1. launch bridge node.

```bash
ros:$ roslaunch fiware_ros_turtlebot3_bridge fiware_ros_turtlebot3_bridge.launch
```

* confirm that the bridge node connected MQTT broker successfully.

```
[INFO] [1531975302.196302]: [fiware_ros_turtlebot3_bridge.base:CmdBridge._on_connect] connected to mqtt broker, status=0
[INFO] [1531975302.204255]: [fiware_ros_turtlebot3_bridge.base:AttrsBridge._on_connect] connected to mqtt broker, status=0
```

if `status` is not 0, `mqtt.yaml` and/or `ca.crt` is invalid.

#### launch node to operate turtlebot3

1. ssh to turtlebot3.
1. launch operator node.

```bash
ros:$ roslaunch fiware_ros_turtlebot3_operator fiware_ros_turtlebot3_operator.launch
```

#### start turtlebot3

1. ssh to turtlebot3.
1. launch turtlebot3 robot

```bash
ros:$ roslaunch turtlebot3_bringup turtlebot3_robot.launch
```

## operate turtlebot3 using gamepad

#### prepare raspberry pi connected to gamepad

1. ssh to raspberry pi connected to gamepad.
1. clone repositories of gamepad controller from github.
    * https://github.com/tech-sketch/fiware-gamepad-controller.git

#### setup cert file to connect MQTT broker of fiware

1. copy the `ca.crt` from local to raspberry pi

```bash
$ scp ${PJ_ROOT}/secrets/ca.crt ${user}@${raspberry_pi}:${repogitory_root}/secrets/ca.crt
```

#### setup the parameters of MQTT broker

1. start a terminal on Ubuntu desktop.
1. setup `mqtt.yaml`

```bash
raspberrypi:$ ls conf/pxkwcr-azure.yaml.template conf/pxkwcr-azure.yaml
raspberrypi:$ vi conf/pxkwcr-azure.yaml
```

#### install python libraries for bridge node

1. install python libraries for bridge node

```bash
raspoberypi:$ pip install -r requirements/common.txt
```

#### run main program

1. run program

```bash
raspberrypi$ ./main.py pxkwcr-azure
```

* confirm that the raspberry pi connected MQTT broker successfully.

```
2018/07/19 14:20:12 [   INFO] src.controller - connected mqtt broker[mqtt.cloudconductor.jp:8883], response_code=0
```

if `response_code` is not 0, `pxkwcr-azure.yaml` and/or `ca.crt` is invalid.

#### control turtlebot3

1. press any button of gamepad.
1. turtlebot3 should move according the pressed button.

## operate turtlebot3 using web controller

#### open web controller
change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/container-centric-fiware-demonstration"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/container-centric-fiware-demonstration
```

In [ ]:
source ${PJ_ROOT}/docs/azure_aks/env

In [ ]:
open https://api.${DOMAIN}/controller/web/

#### confirm basic auth username & password

In [ ]:
cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.basic_auths | map(select(.allowed_paths[] | contains ("/controller/web"))) | .[0].username' -r

In [ ]:
cat ${PJ_ROOT}/secrets/auth-tokens.json | jq '.basic_auths | map(select(.allowed_paths[] | contains ("/controller/web"))) | .[0].password' -r

#### control turtlebot3

1. login the opened page using above user & password.
1. press any button of web controller.
1. turtlebot3 should move according the pressed button.